# Preparation

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"edwardher","key":"b497f5aa60a92a8e3c2f02a3b7efc50c"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

# Make directory named kaggle and copy kaggle.json file there.


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

# Change the permissions of the file.


In [ ]:
! kaggle datasets list 

#- That's all ! You can check if everything's okay by running this command.

# Load Data

In [ ]:
! kaggle datasets download -d openfoodfacts/world-food-facts

 97% 105M/109M [00:01<00:00, 93.1MB/s]
100% 109M/109M [00:01<00:00, 97.4MB/s]


In [ ]:
! mkdir train
! unzip world-food-facts.zip -d train


mkdir: cannot create directory ‘train’: File exists
Archive:  world-food-facts.zip
  inflating: train/en.openfoodfacts.org.products.tsv  


# 연습2 - 데이터에 감 잡기

이번에는 인터넷에서 직접 데이터를 가져오자. 데이터와 자료를 공유해준 https://github.com/justmarkham 에게 특별히 감사를 전한다.

### 단계 1. 필요한 라이브러리를 가져온다.

In [ ]:
import pandas as pd

### 단계 2. 이 [주소](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv) 에서 필요한 데이터를 가져온다.

In [ ]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'


### 단계 3. chipo라는 변수에 저장한다.

In [ ]:
chipo = pd.read_csv(url, sep = '\t')

### 단계 4. 첫 10개 행을 본다.

In [ ]:
chipo.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


### 단계 5. 데이터 안에 몇 개의 관측치가 들어 있나?

In [ ]:
# 해법 1

chipo.shape[0]


4622

In [ ]:
# 해법 2

chipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


### 단계 6. 데이터에 열columns은 몇 개인가?

In [ ]:
chipo.shape[1]

5

### 단계 7. 모든 열의 이름을 출력하라.

In [ ]:
chipo.columns

Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')

### 단계 8. 데이터는 어떻게 색인되어 있나?

In [ ]:
chipo.index

RangeIndex(start=0, stop=4622, step=1)

### 단계 9. 어떤 물건을 가장 많이 구매하나?

In [ ]:
chipo['item_name'].value_counts()

c = chipo.groupby('item_name')
c = c.sum()
c = c.sort_values(['quantity'], ascending=False)
c.head(1)

,order_id,quantity
item_name,,
Chicken Bowl,713926,761


### 단계 10. 가장 많이 구매하는 물건을 총 몇 개나 구매했나?

In [ ]:
temp = (chipo.quantity == chipo['quantity'].max())

print(chipo['quantity'].max())
print(chipo[temp].item_name)

15
3598    Chips and Fresh Tomato Salsa
Name: item_name, dtype: object


### 단계 11. choice_description 열에서 가장 많이 구매한 아이템은 무엇인가?

In [ ]:
chipo['choice_description'].value_counts()

[Diet Coke]                                                                                                                134
[Coke]                                                                                                                     123
[Sprite]                                                                                                                    77
[Fresh Tomato Salsa, [Rice, Black Beans, Cheese, Sour Cream, Lettuce]]                                                      42
[Fresh Tomato Salsa, [Rice, Black Beans, Cheese, Sour Cream, Guacamole, Lettuce]]                                           40
                                                                                                                          ... 
[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Guacamole]]                                                             1
[Roasted Chili Corn Salsa (Medium), [Cheese, Sour Cream]]                                                      

### 단계 12. 모두 합쳐서 몇 개의 아이템을 구매했나?

In [ ]:
chipo['quantity'].sum()

4972

### 단계 13. 아이템 가격item_price을 float 자료형으로 바꿔라

#### 단계 13.a. 아이템 가격의 자료형을 확인하자.

In [ ]:
chipo.item_price.dtype

dtype('O')

#### 단계 13.b. 람다 함수를 만들어서 아이템 가격을 바꿔보자.

In [ ]:
# dollarizer = lambda x: float(x[1:-1])
# chipo.item_price = chipo.item_price.apply(dollarizer)

chipo.item_price = chipo['item_price'].apply(lambda x:float(x[1:]))

#### 단계 13.c. 다시 아이템 가격의 자료형을 확인하자.

In [ ]:
chipo.item_price.dtype
chipo.quantity.dtype

dtype('int64')

### 단계 14. 데이터에 나온 기간 동안 수익revenue은 얼마인가?

In [ ]:

chipo['revenue'] = chipo['quantity']*chipo['item_price']
chipo['revenue'].sum()

39237.02

### 단계 15. 이 기간 동안 얼마나 많은 구매가 이루어졌나?

In [ ]:
# orders = len(chipo.order_id.value_counts())

orders = chipo.order_id.value_counts().count()
orders

1834

### 단계 16. 구매 당 평균 수익은 얼마인가?

In [ ]:
# 해법 1

order_grouped = chipo.groupby(by=['order_id']).sum()
order_grouped

#order_grouped

,quantity,item_price,revenue
order_id,,,
1,4,11.56,11.56
2,2,16.98,33.96
3,2,12.67,12.67
4,2,21.00,21.00
5,2,13.70,13.70
...,...,...,...
1830,2,23.00,23.00
1831,3,12.90,12.90
1832,2,13.20,13.20


In [ ]:
# 해법 2

chipo.groupby(by=['order_id']).sum().mean()['revenue']


21.394231188658654

### 단계 17. 팔린 서로 다른 아이템의 종류는 몇 가지인가?

In [ ]:
chipo.item_name.value_counts().count

50